In [21]:
# подключение библиотек
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import ttest_ind
from scipy.stats import pearsonr


In [22]:
# Объединение данных в один dataframe
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]
data = pd.DataFrame({'Petersburg': pd.Series(petersburg), 'Magadan': pd.Series(magadan)})

data

,Petersburg,Magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,NaN,0.0689


В данных присутствует пропуск - выборка по Петербургу оказалась меньше. Заполним пропуск средним значением

In [23]:
data.fillna({'Petersburg': data['Petersburg'].mean()}, inplace=True)
data

,Petersburg,Magadan
0,0.097400,0.1033
1,0.135200,0.0915
2,0.081700,0.0781
3,0.101600,0.0685
4,0.096800,0.0677
5,0.106400,0.0697
6,0.105000,0.0764
7,0.103443,0.0689


Проверим данные на нормальность распределения для последующего выбора статистического теста

In [24]:
# Зададим уровень значимости
alpha = 0.05

# Сформулируем гипотезу о нормальности распределения данных
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'
# применим тест Шапиро-Уилка

_, p = shapiro(data)
print('p=%.3f' % p)

print( H0 if p > alpha else Ha)

p=0.100
Данные распределены нормально


Так как данные распределены нормально, а зависимая переменная (диаметр мидий) является количественной и совокупности разные (2 разных города), объем выборки менее 30,
для статистического теста возможно применить независимый Т-тест. Корреляцию проверим с помощью коэффициента Пирсона 

In [28]:
# Проводим тест на корреляцию
corr, p_corr = pearsonr(data['Petersburg'], data['Magadan'])

In [26]:
if p_corr>alpha:
    print(f"{p_corr} > {alpha}. Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными.")
else:
    print(f"{p_corr} <= {alpha}. Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными.")

0.5594098062056255 > 0.05. Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными.


In [29]:
# сформулируем гипотезу на сравнение выборок и проведем сравнение
H0_t = 'Нет значимой разницы между диаметрами мидий в разных городах.'
Ha_t = 'Есть значимая разница между диаметрами мидий в разных городах.'

In [34]:
def t_test(df):
    print('\n' + "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(df['Petersburg'], df['Magadan'], equal_var=True)

    p_t = round(test_results[1], 3)

    if p_t>alpha:
        print(f"{p_t} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0_t}")
    else:
        print(f"{p_t} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha_t}")

t_test(data)


*** Результаты независимого T-теста ***
0.003 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между диаметрами мидий в разных городах.
